In [1]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

sys.path.append('../Runs')
from train_QTable import optimal_strategy, run
# from helper.Q_learning import QLearn, round_custombase
from helper.manage_orderbooks import OrderbookEpisodesGenerator, plot_episode
from helper.orderbook_trader import OrderbookTradingSimulator
from agents.QTable_Agent import QTable_Agent

In [4]:
inputfile='/home/axel/data/small/obs_2016-11_USDT_BTC_maxVol100.dict'
# experiment_name='1611_USDTBTC_Qtable_100vol10_60T4___1datapoint'
# outputfolder='experiments'
# outputfile_model=os.path.join(outputfolder, experiment_name, 'model', experiment_name)
outfile_agent='q_10_aggr.json'
outfile_samples='q_10_aggr.csv'
volume=100
volume_intervals=10
decision_points=4
period_length=15
action_min=-0.4
action_max=1.0
action_count=15

state_variables=['volume','time']
actions = list(np.linspace(action_min, action_max, num=action_count))

episodes_train = OrderbookEpisodesGenerator(filename=inputfile,
                                            episode_length=decision_points*period_length)
# data = list(episodes_train[:20])
# pickle.dump( data, open( "cached_episodes.p", "wb" ) )
data = pickle.load( open( "cached_episodes.p", "rb" ) )
print(data[0][0].timestamp)

agent = optimal_strategy(traingdata=data[:1], V=volume, T=decision_points,
                          period_length=period_length, vol_intervals=volume_intervals,
                          actions=actions, state_variables=state_variables,
                          outfile_agent=outfile_agent, outfile_samples=outfile_samples)


  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2016-11-08T10:00
RL-Type: <class 'agents.QTable_Agent.QTable_Agent'>




1it [00:24, 24.54s/it]
 25%|██▌       | 1/4 [00:24<01:13, 24.54s/it]
0it [00:00, ?it/s]

Saved: 'q_10_aggr.json'
Saved: 'q_10_aggr.csv'



1it [00:20, 20.47s/it]
 50%|█████     | 2/4 [00:45<00:46, 23.32s/it]
0it [00:00, ?it/s]

Saved: 'q_10_aggr.json'
Saved: 'q_10_aggr.csv'



1it [00:21, 21.54s/it]
 75%|███████▌  | 3/4 [01:06<00:22, 22.79s/it]
0it [00:00, ?it/s]

Saved: 'q_10_aggr.json'
Saved: 'q_10_aggr.csv'



1it [00:03,  3.68s/it]
100%|██████████| 4/4 [01:10<00:00, 17.06s/it]

Saved: 'q_10_aggr.json'
Saved: 'q_10_aggr.csv'


In [5]:
test = QTable_Agent.load(infile_agent='q_10_aggr.json', infile_samples="q_10_aggr.csv")


In [7]:
test.samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 465 entries, 0 to 464
Data columns (total 10 columns):
volume            465 non-null float64
time              465 non-null float64
action            465 non-null float64
action_idx        465 non-null int64
cost              465 non-null float64
avg               465 non-null float64
initial_center    465 non-null float64
timestamp         465 non-null object
volume_n          465 non-null float64
time_n            465 non-null float64
dtypes: float64(8), int64(1), object(1)
memory usage: 40.0+ KB


In [ ]:
agent.samples

In [ ]:
agent.heatmap_Q(show_traces=True, show_minima_count=False)

In [ ]:
models = ['q_10.json', 'q_10_aggr.json', 'q_10_interp.json', 'q_5.json', 'q_5_interp.json']
costs_insample = None
for m, file in enumerate(models):
    agent = QTable_Agent.load(file)
    print(m, file[:-5])
    if m < len(models)-1:
        costs_insample = agent.evaluate(testdata=data[:10], name=file[:-5], verbose=False, costs=costs_insample)
    else:
        costs_insample = agent.evaluate(testdata=data[:10], name=file[:-5], verbose=False, costs=costs_insample, evaluate_actions=[0.1, 0.2, 0.3, 0.4, 0.5])

display(costs_insample)
display(costs_insample.mean())
best_static_action = costs_insample.mean()[len(models):].argmin()
print("Best S&L action: {}".format(best_static_action))

In [ ]:
models = ['q_10.json', 'q_10_aggr.json', 'q_10_interp.json', 'q_5.json', 'q_5_interp.json']
costs = None
for m, file in enumerate(models):
    agent = QTable_Agent.load(file)
    print(m, file[:-5])
    agent.heatmap_Q()
    if m < len(models)-1:
        costs = agent.evaluate(testdata=data[10:20], name=file[:-5], verbose=False, costs=costs)
    else:
        costs = agent.evaluate(testdata=data[10:20], name=file[:-5], verbose=False, costs=costs, evaluate_actions=[best_static_action])

display(costs)
display(costs.mean())
print("Best S&L action: {}".format(best_static_action))

In [ ]:
costs_insample = agent.evaluate(testdata=data[:10], verbose=False, costs=costs_insample, name='v10')  # , evaluate_actions=[0.1, 0.2, 0.3, 0.4, 0.5])

display(costs_insample)
display(costs_insample.mean())
best_static_action = costs_insample.mean()[2:].argmin()
print("Best S&L action: {}".format(best_static_action))

In [ ]:
#costs = None
costs = agent.evaluate(testdata=data[10:20], verbose=False, name='v10', costs=costs)  # , evaluate_actions=[best_static_action])

display(costs)
display(costs.mean())

In [ ]:
idx = 4
plot_episode(data[idx], volume=100, limits=[0.2])
plot_episode(data[idx], volume=100, limits=[0.4]*4, intervals=5)

In [ ]:
display(costs.iloc[:8,:])
display(costs.iloc[:8,:].mean())

In [ ]:

plot_experiment_results(costs_insample.iloc[:,:], showfliers=True, hline='q_10', ylim=(0, 0.5))

In [ ]:
plot_experiment_results(costs.iloc[:8,:], showfliers=True, hline='v10_interp')